# Preparation

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          ###################                                 57.8%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [3]:
def pull_model(local_llm):
    !ollama pull local_llm

In [4]:
def start_model(local_llm):
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [5]:
%%capture --no-stderr
%pip install -U rouge-score nltk bert-score scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [6]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# response: biased or non-biased
bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

2024-08-13 16:09:46.026689: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 16:09:46.026902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 16:09:46.199202: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


# Bias detection

## Dati

https://www.kaggle.com/datasets/timospinde/mbic-a-media-bias-annotation-dataset (vedi anche paper di riferimento MBIC)

In [8]:
df = pd.read_excel("/kaggle/input/mbic-a-media-bias-annotation-dataset/labeled_dataset.xlsx")
#/kaggle/input/mbic-a-media-bias-annotation-dataset/labeled_dataset.xlsx
#/content/drive/MyDrive/predictions/labeled_dataset.xlsx

print(len(df))

print(df["Label_bias"].unique()) #ci sono tre valori, i no-agreement come li gestiamo?
# eliminiamo i no-agreement (non c'è accordo che siano biased o non-biased).

df = df[df["Label_bias"] != "No agreement"]

df_biased = df[df["Label_bias"] == "Biased"]

print(len(df))
print(len(df_biased)) #1000 tesi biased

1700
['Biased' 'Non-biased' 'No agreement']
1551
1018


/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
#all'incirca 1551 testi di test, news biased e non biased

# Biased = 0
# Non-biased = 1

x_test = df["sentence"]
y_test_pre = df["Label_bias"] #Biased #Non-biased

print(len(x_test))
print(len(y_test_pre))

label_biased = df[df["Label_bias"]=="Biased"] #Label 0
label_non_biased = df[df["Label_bias"]=="Non-biased"] #Label 1
print(f"Label con valore Biased: {len(label_biased)}")
print(f"Label con valore Non-biased: {len(label_non_biased)}")

1551
1551
Label con valore Biased: 1018
Label con valore Non-biased: 533


In [ ]:
y_test = [0 if label=="Biased" else 1 for label in y_test_pre]

print(f"Label con valore Biased: {y_test.count(0)}")
print(f"Label con valore Non-biased: {y_test.count(1)}")

## Test

In [ ]:
examples = [
    {"input": "I'm not racist, but [group] are just not as smart as us.", "output": "biased"},
    {"input": "Women should know their place.", "output": "biased"},
    {"input": "She excelled in her exams due to her dedication and hard work.", "output": "no-biased"},
    {"input": "People from that country are lazy and unproductive.", "output": "biased"},
    {"input": "He was selected for the role based on his qualifications and experience.", "output": "non-biased"},
    {"input": "Students from that school always perform poorly.", "output": "biased"},
    {"input": "The decision was made after a thorough review of all the candidates.", "output": "no-biased"},
    {"input": "He's not suitable for the job because he's too old.", "output": "biased"},
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

def create_prompt(shot):
    example_prompt  = ChatPromptTemplate.from_messages(
        [
            ("user", "{input}"),
            ("assistant", "{output}"),
        ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt ,
        examples=examples[:shot], #:2 two-shot, #4: four-shot #8 eight-shot
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You will be provided with a text, and your task is to classify it into
                        one of two categories: "biased" or "non-biased." A text is considered "biased"
                        if it expresses a personal opinion, uses emotional or partial language, or promotes
                        a particular perspective at the expense of others. A text is considered "non-biased"
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
             few_shot_prompt,
            ("user", "{input}"),
        ] if shot>0 else
        [
            ("system", """You will be provided with a text, and your task is to classify it into
                        one of two categories: "biased" or "non-biased." A text is considered "biased"
                        if it expresses a personal opinion, uses emotional or partial language, or promotes
                        a particular perspective at the expense of others. A text is considered "non-biased"
                        if it presents facts in a neutral and objective manner, without evident influences.
                        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS."""),
            ("user", "{input}"),
        ]
    )
    return prompt

In [ ]:
# response: biased or non-biased

def bias_det(llm, shot):
    prompt_final = create_prompt(shot)
    bias_det_chain = prompt_final | llm
    return bias_det_chain

In [ ]:
# Conferma label
# Label 0: biased
# Label 1: non-biased

def predict(llm,x_test,shot,encoder):
    y_pred = []
    y_pred_label = []
    if encoder:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
    else:
        chain = bias_det(llm,shot)
    for x in tqdm(x_test):
        if encoder:
            answer = bias_detection(x)[0]["label"]
        else:
            answer = chain.invoke({"input": x})
        if "non-biased" in answer.lower(): y_pred.append(1)
        else: y_pred.append(0)
        y_pred_label.append(answer) #per verificare che le risposte siano sensate
    return y_pred, y_pred_label

# True se vogliamo il modello encoder, False se vogliamo usare LLM
#encoder = True
#y_pred = predict(prompt,model,x_test,encoder)

In [ ]:
import time
import json

models = ["llama3.1","gemma2","mistral"]
shots = [0,2,4,8]


def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

for model in models:
    if model == "encoder":
        y_pred, y_pred_label = predict(None,x_test,0,True)
        write_file(f"/content/drive/MyDrive/predictions/prediction_bias_encoder.json", y_pred)
    else:
        start_ollama()
        pull_model(model)
        start_model(model)
        time.sleep(500)
        llm = Ollama(model=model, temperature=0)
        for shot in shots:
            y_pred, y_pred_label = predict(llm,x_test,shot,False)
            if model=="llama3.1":
                write_file(f"/content/drive/MyDrive/predictions/prediction_bias_llama31_{shot}_shot.json", y_pred)
                write_file(f"/content/drive/MyDrive/predictions/prediction_label_bias_llama31_{shot}_shot.json", y_pred_label)
            else:
                write_file(f"/content/drive/MyDrive/predictions/prediction_bias_{model}_{shot}_shot.json", y_pred)
                write_file(f"/content/drive/MyDrive/predictions/prediction_label_bias_{model}_{shot}_shot.json", y_pred_label)

In [ ]:
# Valutazione del modello
#Label 0: biased
#Label 1: non biased
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

y_pred = load_json("/content/drive/MyDrive/predictions/prediction_bias_0_shot.json")

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)


#dict_to_write = {"class_report":class_report, "0_0":int(conf_matrix[0][0]), "0_1":int(conf_matrix[0][1]), "1_0":int(conf_matrix[1][0]), "1_1":int(conf_matrix[1][1])}
#write_file("/kaggle/working/test_bias_mistral_zero_shot.json", dict_to_write)

#y_pred = [0,1,0]
#write_file("/content/drive/MyDrive/predictions/prova.json", y_pred)

# Debiasing

In [10]:
x_test = df_biased["sentence"]
print(len(x_test))

1018


In [11]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes. \n
         <|eot_id|><|start_header_id|>user<|end_header_id|>
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    debiasing = prompt | local_llm | StrOutputParser()
    return debiasing

In [12]:
def debias(llm,x_test):
    x_pred = []
    chain = debiasing_answer(llm)
    for x in tqdm(x_test):
        answer = chain.invoke({"text": x})
        x_pred.append(answer) 
    return x_pred

In [ ]:
import time
import json


models = ["llama3.1","gemma2","mistral"]
#shots = [0,2,4,8]

#/kaggle/working/
#/content/drive/MyDrive/predictions/


def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

for model in models:
    start_ollama()
    pull_model(model)
    start_model(model)
    time.sleep(500)
    llm = Ollama(model=model, temperature=0)
    x_pred = debias(llm,x_test)  
    if model=="llama3.1":
        write_file(f"/kaggle/working/debias_llama31.json", x_pred)
    else:
        write_file(f"/kaggle/working/debias_{model}.json", x_pred)

Error: listen tcp 127.0.0.1:11434: bind: address already in use


[GIN] 2024/08/13 - 17:05:36 | 200 |      28.351µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ [GIN] 2024/08/13 - 17:05:37 | 200 |  332.686536ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠹ pulling manifest 
Error: pull model manifest: file does not exist
[GIN] 2024/08/13 - 17:05:37 | 200 |      24.272µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/08/13 - 17:05:37 | 200 |   23.451918ms |       127.0.0.1 | POST     "/api/show"


⠙ ⠹ time=2024-08-13T17:05:37.539Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe gpu=GPU-8716c71e-b51e-fa6a-2c7c-09f507fa9cdc parallel=4 available=15720382464 required="6.2 GiB"
time=2024-08-13T17:05:37.540Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[14.6 GiB]" memory.required.full="6.2 GiB" memory.required.partial="6.2 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[6.2 GiB]" memory.weights.total="4.7 GiB" memory.weights.repeating="4.3 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-08-13T17:05:37.541Z level=INFO source=server.go:393 msg="starting llama server" cmd="/tmp/ollama958192415/runners/cuda_v11/ollama_llama_server --model /root/.ollama/model

INFO [main] build info | build=1 commit="1e6f655" tid="134135467356160" timestamp=1723568737
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="134135467356160" timestamp=1723568737 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="34587" tid="134135467356160" timestamp=1723568737


llama_model_loader: loaded meta data with 29 key-value pairs and 292 tensors from /root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str

INFO [main] model loaded | tid="134135467356160" timestamp=1723568741
[GIN] 2024/08/13 - 17:05:41 | 200 |  3.902654251s |       127.0.0.1 | POST     "/api/generate"


time=2024-08-13T17:05:41.164Z level=INFO source=server.go:632 msg="llama runner started in 3.62 seconds"


## Evaluation

In [ ]:
# read x_pred

x_pred = load_json(f"/kaggle/working/debias_{model}.json")

In [ ]:
classificatore di supporto

classifier_model =  "llama3.1"
shot_classifier_model = 0

#1 = non biased
#0 = biased

def num_non_biased(x_pred, classifier_model, shot_classifier_model):
    start_ollama()
    pull_model(classifier_model)
    start_model(classifier_model)
    time.sleep(500)
    y_pred = predict(classifier_model,x_pred,shot_classifier_model,False)
    return y_pred.count(1)
    
total_debiased = num_non_biased(x_pred,classifier_model,shot_classifier_model)

https://www.nltk.org/api/nltk.translate.bleu

In [ ]:
# calcolo bleu

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothie = SmoothingFunction().method4 #capisci cos'è smoothing function e method4
bleu_scores = []

# Itera sulle frasi
for test, pred in tqdm(zip(x_test, x_pred)):
    bleu_score = sentence_bleu([test.split()], pred.split(), smoothing_function=smoothie)
    bleu_scores.append(bleu_score)

In [ ]:
# calcolo rouge

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

# Itera sulle frasi
for test, pred in tqdm(zip(x_test, x_pred)):
    scores = scorer.score(test, pred)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Calcola la media dei risultati
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

In [ ]:
# calcolo bert_score

# Calcolo del BERTScore per tutte le coppie di frasi
P, R, F1 = score(x_pred, x_test, lang='en', verbose=True)